In [1]:
from os.path import dirname, basename

import pandas as pd

from inpainting.load import load_dataframe
from inpainting.utils import get_paths

In [2]:
dataset = 'DAVIS'
results_dirs = get_paths(f'../results/ts/{dataset}/*')

times_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_df = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(times_df, results_df, on=['model', 'sequence_name', 'frame_id'])
df

,frame_id,inference_time,model,sequence_name,object_coverage,background_coverage,object_sequence_name,object_id
0,0,25.491392,ts_dilation_3_3,bike-packing_bike-packing_001,0.971456,0.936966,bike-packing,1
1,1,28.357183,ts_dilation_3_3,bike-packing_bike-packing_001,0.967389,0.939151,bike-packing,1
2,2,26.060255,ts_dilation_3_3,bike-packing_bike-packing_001,0.970926,0.936416,bike-packing,1
3,3,27.517120,ts_dilation_3_3,bike-packing_bike-packing_001,0.970808,0.936302,bike-packing,1
4,4,26.227327,ts_dilation_3_3,bike-packing_bike-packing_001,0.990520,0.928603,bike-packing,1
...,...,...,...,...,...,...,...,...
34430,66,26.165056,ts_no_dilation,walking_walking_003,0.236302,0.944671,walking,3
34431,67,26.874687,ts_no_dilation,walking_walking_003,0.241014,0.937395,walking,3
34432,68,24.073088,ts_no_dilation,walking_walking_003,0.237284,0.933758,walking,3
34433,69,24.360384,ts_no_dilation,walking_walking_003,0.259947,0.919245,walking,3


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['inference_time'] = df['inference_time']
metrics['object_coverage'] = df['object_coverage']
metrics['background_coverage'] = df['background_coverage']
metrics.groupby('model').mean()

,inference_time,object_coverage,background_coverage
model,,,
ts_dilation_3_3,24.038739,0.777306,0.964055
ts_dilation_3_5,29.677976,0.794485,0.955769
ts_dilation_5_3,26.993573,0.801514,0.951558
ts_dilation_5_5,31.617822,0.824154,0.934430
ts_no_dilation,24.019950,0.728417,0.975611
